In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import precision_score, confusion_matrix, classification_report

In [ ]:
#Returns just one month but we need 3 months.


df = pd.read_csv('/home/jensk/code/JensKlug/zillows_real_estate/raw_data/HouseTS.csv')

df = df.sort_values(['city', 'zipcode', 'date']).reset_index(drop=True)
df['date'] = pd.to_datetime(df['date'])

df['year_month'] = df['date'].dt.to_period('M')

last_months = df.groupby('zipcode')['year_month'].max().reset_index()
last_rows = df.merge(last_months, on=['zipcode', 'year_month'], how='inner')

df_marked = df.merge(last_months, on=['zipcode', 'year_month'], how='left', indicator=True)

# Step 3: Keep only rows NOT in the last month
df_filtered = df_marked[df_marked['_merge'] == 'left_only'].drop(columns=['_merge'])

train = df_filtered.copy()

grouped_train = train.groupby(['city', 'zipcode'])
grouped_test = last_rows.groupby(['city', 'zipcode'])
'''

In [45]:
df = pd.read_csv('/home/jensk/code/JensKlug/zillows_real_estate/raw_data/HouseTS.csv')
df = df.sort_values(['city', 'zipcode', 'date']).reset_index(drop=True)
df['date'] = pd.to_datetime(df['date'])
df['year_month'] = df['date'].dt.to_period('M')
df = df.sort_values(['city', 'zipcode', 'year_month']).reset_index(drop=True)

# Get the last 3 year_months for each zipcode
last_3_months = (
    df.groupby('zipcode')
    .apply(lambda x: x.nlargest(3, 'year_month'))
    .reset_index(drop=True)
)

# Create a multi-index of zipcode + year_month for the last 3 months
last_3_keys = last_3_months[['zipcode', 'year_month']].drop_duplicates()

# Merge to flag rows in last 3 months
df_merged = df.merge(last_3_keys, on=['zipcode', 'year_month'], how='left', indicator=True)

# Filter out last 3 months
train = df_merged[df_merged['_merge'] == 'left_only'].drop(columns=['_merge'])

# Optional: Create groups if needed
grouped_train = train.groupby(['city', 'zipcode'])
grouped_test = last_3_months.groupby(['city', 'zipcode'])

/tmp/ipykernel_1655/471952232.py:10: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.nlargest(3, 'year_month'))


In [ ]:
#
'''
df = pd.read_csv('/home/jensk/code/JensKlug/zillows_real_estate/raw_data/HouseTS.csv')

df = df.sort_values(['city', 'zipcode', 'date']).reset_index(drop=True)

df['date'] = pd.to_datetime(df['date'])

df['year_month'] = df['date'].dt.to_period('M')

df = df.sort_values(['city', 'zipcode', 'year_month']).reset_index(drop=True)

last_3_rows = (
    df.groupby('zipcode', group_keys=False)
    .apply(lambda x: x.nlargest(3, 'year_month'))
)

# Step 2: Create a set of (zipcode, year_month) to exclude
exclude_set = set(zip(last_3_rows['zipcode'], last_3_rows['year_month']))

# Step 3: Filter out these from the full dataset
train = df[~df[['zipcode', 'year_month']].apply(tuple, axis=1).isin(exclude_set)]

# Sanity check
print(f"Total rows in df: {len(df)}")
print(f"Rows in train (should be full minus last 3 months per zipcode): {len(train)}")
print(f"Rows in test (last 3 months per zipcode): {len(last_3_rows)}")
'''

/tmp/ipykernel_1655/51279733.py:13: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.nlargest(3, 'year_month'))


Total rows in df: 884092
Rows in train (should be full minus last 3 months per zipcode): 865414
Rows in test (last 3 months per zipcode): 18678


In [54]:
#train['zipcode']
last_3_months['zipcode']

0         1431
1         1431
2         1431
3         1432
4         1432
         ...  
18673    98685
18674    98685
18675    98686
18676    98686
18677    98686
Name: zipcode, Length: 18678, dtype: int64

In [40]:
grouped_train.head(3).sort_values(['zipcode', 'date'], ascending=(True, True))['date']

41038    2012-03-31
41039    2012-04-30
41040    2012-05-31
41180    2012-03-31
41181    2012-04-30
            ...    
614009   2012-04-30
614010   2012-05-31
614150   2012-03-31
614151   2012-04-30
614152   2012-05-31
Name: date, Length: 18678, dtype: datetime64[ns]

In [35]:
#grouped_train.head(3)
grouped_test.head(3).sort_values(['zipcode', 'date'], ascending=(True, True))['zipcode']

2         1431
1         1431
0         1431
5         1432
4         1432
         ...  
18673    98685
18672    98685
18677    98686
18676    98686
18675    98686
Name: zipcode, Length: 18678, dtype: int64

In [ ]:
'''
df = pd.read_csv('/home/jensk/code/JensKlug/zillows_real_estate/raw_data/HouseTS.csv')

df = df.sort_values(['zipcode', 'date']).reset_index(drop=True)
df['date'] = pd.to_datetime(df['date'])

df['year_month'] = df['date'].dt.to_period('M')

last_3_months = (
    df.sort_values(['zipcode', 'date'], ascending=[True, False])
      .groupby('zipcode')
      .head(3)
)

# Optional: sort for readability
last_3_months = last_3_months.sort_values(['zipcode', 'date']).reset_index(drop=True)
last_rows = df.merge(last_3_months, on=['zipcode', 'year_month'], how='inner')

df_marked = df.merge(last_3_months, on=['zipcode', 'year_month'], how='left', indicator=True)

# Step 3: Keep only rows NOT in the last month
df_filtered = df_marked[df_marked['_merge'] == 'left_only'].drop(columns=['_merge'])

train = df_filtered.copy()

grouped_train = train.groupby(['zipcode'])
grouped_test = last_rows.groupby(['zipcode'])
'''

In [16]:
#df.head(3)
last_3_months
#last_months
#last_rows
#df_filtered.iloc[120:151] # Here we see that December is missing in 2023. :)

,date,median_sale_price,median_list_price,median_ppsf,median_list_ppsf,homes_sold,pending_sales,new_listings,inventory,median_dom,...,Median Rent,Median Home Value,Total Labor Force,Unemployed Population,Total School Age Population,Total School Enrollment,Median Commute Time,price,city_full,year_month
0,2023-12-31,461500.0,404900.0,200.601023,210.568245,8.0,11.0,14.0,8.0,57.5,...,987.0,328700.0,1921.0,97.0,3170.0,3170.0,1541.0,431806.192745,Boston-Cambridge-Newton,2023-12
1,2023-11-30,469000.0,398888.0,200.601023,210.409745,10.0,9.0,11.0,9.0,32.0,...,987.0,328700.0,1921.0,97.0,3170.0,3170.0,1541.0,431098.634112,Boston-Cambridge-Newton,2023-11
2,2023-10-31,462500.0,450000.0,219.758478,191.236878,16.0,10.0,9.0,7.0,22.0,...,987.0,328700.0,1921.0,97.0,3170.0,3170.0,1541.0,429997.675464,Boston-Cambridge-Newton,2023-10
3,2023-12-31,492500.0,489900.0,343.672800,287.861374,14.0,16.0,18.0,6.0,17.5,...,1333.0,397500.0,5099.0,296.0,7887.0,7887.0,3905.0,508515.970387,Boston-Cambridge-Newton,2023-12
4,2023-11-30,476000.0,499900.0,343.023256,291.360987,17.0,16.0,19.0,8.0,17.0,...,1333.0,397500.0,5099.0,296.0,7887.0,7887.0,3905.0,505611.783019,Boston-Cambridge-Newton,2023-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18673,2023-11-30,640000.0,599000.0,291.176471,287.162162,71.0,75.0,99.0,69.0,27.0,...,1625.0,513000.0,15814.0,497.0,29456.0,29456.0,11629.0,577994.879622,Portland-Vancouver-Hillsboro,2023-11
18674,2023-10-31,640000.0,599475.0,297.304440,294.014882,81.0,83.0,106.0,72.0,21.0,...,1625.0,513000.0,15814.0,497.0,29456.0,29456.0,11629.0,577247.107610,Portland-Vancouver-Hillsboro,2023-10
18675,2023-12-31,520200.0,519900.0,299.494207,293.822411,101.0,73.0,72.0,63.0,31.0,...,1677.0,497100.0,10143.0,395.0,19904.0,19904.0,7787.0,569875.467016,Portland-Vancouver-Hillsboro,2023-12
18676,2023-11-30,525000.0,532580.0,309.415584,296.314311,118.0,78.0,84.0,70.0,30.5,...,1677.0,497100.0,10143.0,395.0,19904.0,19904.0,7787.0,569327.047815,Portland-Vancouver-Hillsboro,2023-11


In [32]:
ts_train = pd.DataFrame()

ls = [1,3,6,12]

i = 1

#for i in ls:
ts_train[f'target_profitable_{i}m'] = (grouped_train['price'].shift(-i) > 1.02 * train['price']).astype(int) # We fail to exclude the last three obervastions since with the shaft they are NaN!
ts_train['zipcode'] = grouped_train['zipcode'].shift(0)# .astype('object') # but remains still an integer...
numeric_column_names = train.select_dtypes(include='number').columns # drop the lags of zipcode

for feature in numeric_column_names:
    for lag in range(1, 4):  # 1 to 3 months
        ts_train[f'{feature}_{lag}m'] = grouped_train[f'{feature}'].shift(lag)


ts_test = pd.DataFrame()

#for i in ls:
ts_test[f'target_profitable_{i}m'] = (grouped_test['price'].shift(-i) > 1.02 * last_rows['price']).astype(int) # We fail to exclude the last three obervastions since with the shaft they are NaN!
ts_test['zipcode'] = grouped_test['zipcode'].shift(0)# .astype('object') # but remains still an integer...
numeric_column_names = last_rows.select_dtypes(include='number').columns # drop the lags of zipcode

for feature in numeric_column_names:
    for lag in range(1, 4):  # 1 to 3 months
        ts_test[f'{feature}_{lag}m'] = grouped_test[f'{feature}'].shift(lag)

#ts_test = ts_test.dropna().reset_index(drop=True)

/tmp/ipykernel_1904/871680835.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ts_train[f'{feature}_{lag}m'] = grouped_train[f'{feature}'].shift(lag)
/tmp/ipykernel_1904/871680835.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ts_train[f'{feature}_{lag}m'] = grouped_train[f'{feature}'].shift(lag)
/tmp/ipykernel_1904/871680835.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using

In [34]:
ts_test.shape, ts_test

((6226, 110),
       target_profitable_1m  zipcode  median_sale_price_1m  \
 0                        0    30002                   NaN   
 1                        0    30004                   NaN   
 2                        0    30005                   NaN   
 3                        0    30008                   NaN   
 4                        0    30009                   NaN   
 ...                    ...      ...                   ...   
 6221                     0    34689                   NaN   
 6222                     0    34690                   NaN   
 6223                     0    34691                   NaN   
 6224                     0    34695                   NaN   
 6225                     0    34698                   NaN   
 
       median_sale_price_2m  median_sale_price_3m  median_list_price_1m  \
 0                      NaN                   NaN                   NaN   
 1                      NaN                   NaN                   NaN   
 2             

In [30]:
ts_test = ts_test.dropna()

In [4]:
ts_train['target_profitable_1m'].value_counts()

target_profitable_1m
0    838715
1     39151
Name: count, dtype: int64

In [23]:
ts_train = ts_train.dropna()

In [11]:
ts_train = ts_train.reset_index(drop=True)

In [7]:
X_train = ts_train.drop(columns=[f'target_profitable_{i}m'])
y_train = ts_train[f'target_profitable_{i}m']

In [8]:
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)

In [9]:
X_train_scaled.shape

(859188, 109)

In [25]:
pred = ts_test.drop(columns=[f'target_profitable_{i}m'])

In [31]:
pred.shape, ts_test.shape

((6226, 109), (0, 110))

In [27]:

X_pred_scaled = scaler.transform(pred)

model = SGDClassifier(loss='log_loss')
model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_pred_scaled)

np.savetxt(f"output_{i}mm.csv", y_pred, delimiter=",", header=f"target_-{i}m", comments='')

del y_pred


ValueError: Input X contains NaN.
SGDClassifier does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
zipcode_identifier = ts_test['zipcode']
zipcode_identifier

In [ ]:
#X_test_scaled

In [ ]:
os.getcwd()

In [ ]:
output_1 = pd.read_csv('output_1m.csv')
output_1

In [ ]:
output_2 = pd.read_csv('output_2m.csv')
output_2

In [ ]:
df_combined = pd.concat([zipcode_identifier, output_1, output_2], axis=1)
df_combined

In [ ]:
grouped_result = df_combined.groupby(['zipcode']).mean()
grouped_result.value_counts()

In [ ]:
'''ts_train = pd.DataFrame()
ts_test = pd.DataFrame()

#ls = [1,3,6,12]



#for i in ls:
ts_train['target_profitable_3m'] = (grouped_train['price'].shift(-6) > 1.02 * grouped_train['price'].shift(0)).astype(int) # We fail to exclude the last three obervastions since with the shaft they are NaN!

numeric_column_names = train.select_dtypes(include='number').columns # drop the lags of zipcode

for feature in numeric_column_names:
    for lag in range(1, 4):  # 1 to 3 months
        ts_train[f'{feature}_{lag}m'] = grouped_train[f'{feature}'].shift(lag)

ts_train = ts_train.dropna().reset_index(drop=True)

X_train = ts_train.drop(columns=['target_profitable_3m'])
y_train = ts_train['target_profitable_3m']


ts_test['target_profitable_3m'] = (grouped_test['price'].shift(-6) > 1.02 * grouped_test['price'].shift(0)).astype(int)

numeric_column_names = test.select_dtypes(include='number').columns # drop the lags of zipcode

for feature in numeric_column_names:
    for lag in range(1, 4):  # 1 to 6 months
        ts_test[f'{feature}_{lag}m'] = grouped_test[f'{feature}'].shift(lag)

ts_test = ts_test.dropna().reset_index(drop=True)

X_test = ts_test.drop(columns=['target_profitable_3m'])
y_test = ts_test['target_profitable_3m']

scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = SGDClassifier(loss='log_loss')
model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled)
'''

In [ ]:
'''ts_train = pd.DataFrame()
ts_test = pd.DataFrame()


ls = [1,3,6,12]

for i in ls:
    ts_train[f'target_profitable_{i}m'] = (grouped_train['price'].shift(-i) > 1.02 * grouped_train['price'].shift(0)).astype(int) # We fail to exclude the last three obervastions since with the shaft they are NaN!

    numeric_column_names = train.select_dtypes(include='number').columns # drop the lags of zipcode

    for feature in numeric_column_names:
        for lag in range(1, 4):  # 1 to 3 months
            ts_train[f'{feature}_{lag}m'] = grouped_train[f'{feature}'].shift(lag)

    ts_train = ts_train.dropna().reset_index(drop=True)

    X_train = ts_train.drop(columns=[f'target_profitable_{i}m'])
    y_train = ts_train[f'target_profitable_{i}m']


    ts_test[f'target_profitable_{i}m'] = (grouped_test['price'].shift(-i) > 1.02 * grouped_test['price'].shift(0)).astype(int)

    numeric_column_names = test.select_dtypes(include='number').columns # drop the lags of zipcode

    for feature in numeric_column_names:
        for lag in range(1, 4):  # 1 to 6 months
            ts_test[f'{feature}_{lag}m'] = grouped_test[f'{feature}'].shift(lag)

    ts_test = ts_test.dropna().reset_index(drop=True)

    X_test = ts_test.drop(columns=[f'target_profitable_{i}m'])
    y_test = ts_test[f'target_profitable_{i}m']

    scaler = MinMaxScaler()
    scaler.fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    model = SGDClassifier(loss='log_loss')
    model.fit(X_train_scaled, y_train)

    y_pred = model.predict(X_test_scaled)

    np.savetxt(f"output_{i}m.csv", y_pred, delimiter=",", header=f"target_-{i}m", comments='')

    del y_pred
'''